<div>
<img src=https://www.institutedata.com/wp-content/uploads/2019/10/iod_h_tp_primary_c.svg width="300">
</div>

In [2]:
!pip install regex

distutils: /Users/uchino/anaconda3/include/python3.7m/UNKNOWN
sysconfig: /Users/uchino/anaconda3/include/python3.7m
user = False
home = None
root = None
prefix = None
     |████████████████████████████████| 285 kB 2.7 MB/s eta 0:00:01
distutils: /Users/uchino/anaconda3/include/python3.7m/UNKNOWN
sysconfig: /Users/uchino/anaconda3/include/python3.7m
user = False
home = None
root = None
prefix = None


# Lab 9.2: Web Scraping
INSTRUCTIONS:
- Read the guides and hints then create the necessary analysis and code to find an answer and conclusion for the task below.

# Web Scraping in Python (using BeautifulSoup)

## Scraping Rules
1. **Always** check a website’s **Terms and Conditions** before you scrape it. Be careful to read the statements about legal use of data. Usually, the retrieved data should not be used for commercial purposes.
2. **Do not** request data from the website too aggressively with a program (also known as spamming), as this may break the website. Make sure the program behaves in a reasonable manner (i.e. acts like a human). One request for one webpage per second is good practice.
3. The layout of a website may change from time to time, so make sure to revisit the site and rewrite the code as needed

## Find a Page
Visit the [Fandom](http://fandom.wikia.com) website, find a wikia of your interest and pick a page to work with.

Open a web page with the browser and inspect it.

Hover the cursor on the text and follow the shaded box surrounding the main text.

From the result, check the main text inside a few levels of HTML tags.

In [3]:
## Import Libraries
import regex as re

from urllib.parse import unquote
import urllib3
from bs4 import BeautifulSoup

import warnings
warnings.filterwarnings('ignore')

### Define the content to retrieve (webpage's URL)

In [4]:
# specify the url
quote_page = 'https://bigbangtheory.fandom.com/wiki/Barry_Kripke'

### Retrieve the page
- Require Internet connection

In [5]:
# query the website and return the html to the variable ‘page’
http = urllib3.PoolManager()
r = http.request('GET', quote_page)
if r.status == 200:
    page = r.data
    print('Type of the variable \'page\':', page.__class__.__name__)
    print('Page Retrieved. Request Status: %d, Page Size: %d' % (r.status, len(page)))
else:
    print('Some problem occurred. Request Status: %s' % r.status)

Type of the variable 'page': bytes
Page Retrieved. Request Status: 200, Page Size: 308659


### Convert the stream of bytes into a BeautifulSoup representation

In [6]:
# parse the html using beautiful soup and store in variable `soup`
soup = BeautifulSoup(page, 'html.parser')
print('Type of the variable \'soup\':', soup.__class__.__name__)

Type of the variable 'soup': BeautifulSoup


### Check the content
- The HTML source
- Includes all tags and scripts
- Can be long!

In [7]:
print(soup.prettify()[:1000])

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   Barry Kripke | The Big Bang Theory Wiki | Fandom
  </title>
  <script>
   document.documentElement.className = document.documentElement.className.replace( /(^|\s)client-nojs(\s|$)/, "$1client-js$2" );
  </script>
  <script>
   (window.RLQ=window.RLQ||[]).push(function(){mw.config.set({"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":false,"wgNamespaceNumber":0,"wgPageName":"Barry_Kripke","wgTitle":"Barry Kripke","wgCurRevisionId":350824,"wgRevisionId":350824,"wgArticleId":2273,"wgIsArticle":true,"wgIsRedirect":false,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Characters","Caltech Faculty","Scientists","Physicists","Experimental Physicists","Theoretical Physicists","Particle Physicists","Recurring Characters","Season 2","Season 3","Season 4","Season 5","Season 6","Season 7","Season 8","Season 9","The Big Bang Theory","Kripke","Single","Sheldon

### Check the HTML's Title

In [8]:
print('Title tag :%s:' % soup.title)
print('Title text:%s:' % soup.title.string)

Title tag :<title>Barry Kripke | The Big Bang Theory Wiki | Fandom</title>:
Title text:Barry Kripke | The Big Bang Theory Wiki | Fandom:


### Find the main content
- Check if it is possible to use only the relevant data

In [9]:
article_tag = 'article'
article = soup.find_all(article_tag)[0]
print('Type of the variable \'article\':', article.__class__.__name__)

Type of the variable 'article': Tag


In [10]:
article.text

'\n\n\n    (function () {\n        // we can\'t use mw.config here because it\'s too early so we need to pass this data in different way\n        var videoDetails = {\n            mediaId: "CR0MZ2ZP",\n            impressionsPerSession: 2,\n            isDedicatedForArticle: false,\n        };\n        var hasVideoOnPage = null;\n        var videoBridgeCountries = ["US","GB","CA","AU","DE","SG","MY","VN","ID","PH","TH"];\n\n        function getCookieValue(cookieName) {\n            var cookieSplit = (\'; \' + document.cookie).split(\'; \' + cookieName + \'=\');\n            return cookieSplit.length === 2 ? cookieSplit.pop().split(\';\').shift() : null;\n        }\n\n        function hasMaxedOutPlayerImpressionsInWiki() {\n            var impressionsSoFar = Number(getCookieValue(\'playerImpressionsInWiki\')) || 0;\n            var allowedImpressions = Number(videoDetails.impressionsPerSession);\n\n            if (!hasSeenTheVideoInCurrentSession()) {\n                return false;\n   

### Get some of the text
- Plain text without HTML tags

In [11]:
# show the first 500 characters after removing redundant newlines
print(re.sub(r'\n\n+', '\n', article.text)[:500])


    (function () {
        // we can't use mw.config here because it's too early so we need to pass this data in different way
        var videoDetails = {
            mediaId: "CR0MZ2ZP",
            impressionsPerSession: 2,
            isDedicatedForArticle: false,
        };
        var hasVideoOnPage = null;
        var videoBridgeCountries = ["US","GB","CA","AU","DE","SG","MY","VN","ID","PH","TH"];
        function getCookieValue(cookieName) {
            var cookieSplit = ('; ' + documen


### Find the links in the text

In [12]:
for t in article.find_all('a'):
    print(t)

<a class="image image-thumbnail " href="https://static.wikia.nocookie.net/bigbangtheory/images/e/e7/Plag16.jpg/revision/latest?cb=20190506103936" title="&lt;sup&gt;Adult&lt;/sup&gt;">
<img alt="&lt;sup&gt;Adult&lt;/sup&gt;" class="pi-image-thumbnail" data-image-key="Plag16.jpg" data-image-name="Plag16.jpg" height="415" src="https://static.wikia.nocookie.net/bigbangtheory/images/e/e7/Plag16.jpg/revision/latest/scale-to-width-down/310?cb=20190506103936" srcset="https://static.wikia.nocookie.net/bigbangtheory/images/e/e7/Plag16.jpg/revision/latest/scale-to-width-down/310?cb=20190506103936 1x, https://static.wikia.nocookie.net/bigbangtheory/images/e/e7/Plag16.jpg/revision/latest/scale-to-width-down/619?cb=20190506103936 2x" width="300">
</img></a>
<a class="image image-thumbnail " href="https://static.wikia.nocookie.net/bigbangtheory/images/f/f6/BarryKripke.png/revision/latest?cb=20110225223327" title="&lt;sup&gt;Young Adult&lt;/sup&gt;">
<img alt="&lt;sup&gt;Young Adult&lt;/sup&gt;" class

<a href="/wiki/Debbie_Wolowitz" title="Debbie Wolowitz">Debbie Wolowitz</a>
<a href="/wiki/Priya_Koothrappali" title="Priya Koothrappali">Priya Koothrappali</a>
<a href="/wiki/V._M._Koothrappali" title="V. M. Koothrappali">Dr. V. M. Koothrappali</a>
<a href="/wiki/Mrs._Koothrappali" title="Mrs. Koothrappali">Mrs. Koothrappali</a>
<a href="/wiki/Stephanie_Barnett" title="Stephanie Barnett">Stephanie Barnett</a>
<a href="/wiki/Lucy" title="Lucy">Lucy</a>
<a href="/wiki/Mary_Cooper" title="Mary Cooper">Mary Cooper</a>
<a href="/wiki/Zack_Johnson" title="Zack Johnson">Zack Johnson</a>
<a href="/wiki/Beverly_Hofstadter" title="Beverly Hofstadter">Beverly Hofstadter</a>
<a href="/wiki/Mike_Rostenkowski" title="Mike Rostenkowski">Mike Rostenkowski</a>
<a href="/wiki/Alex_Jensen" title="Alex Jensen">Alex Jensen</a>
<a href="/wiki/Professor_Proton" title="Professor Proton">Professor Proton</a>
<a href="/wiki/Missy_Cooper" title="Missy Cooper">Missy Cooper</a>
<a href="/wiki/Ramona_Nowitzki" tit

<a href="/wiki/Category:Season_3" title="Category:Season 3">Season 3</a>
<a href="/wiki/Category:Season_4" title="Category:Season 4">Season 4</a>
<a href="/wiki/Category:Season_5" title="Category:Season 5">Season 5</a>
<a href="/wiki/Category:Season_6" title="Category:Season 6">Season 6</a>
<a href="/wiki/Category:Season_7" title="Category:Season 7">Season 7</a>
<a href="/wiki/Category:Season_8" title="Category:Season 8">Season 8</a>
<a href="/wiki/Category:Season_9" title="Category:Season 9">Season 9</a>
<a href="/wiki/Category:The_Big_Bang_Theory" title="Category:The Big Bang Theory">The Big Bang Theory</a>
<a href="/wiki/Category:Kripke" title="Category:Kripke">Kripke</a>
<a href="/wiki/Category:Single" title="Category:Single">Single</a>
<a href="/wiki/Category:Sheldon%27s_Mortal_Enemies" title="Category:Sheldon's Mortal Enemies">Sheldon's Mortal Enemies</a>
<a href="/wiki/Category:Ph.D." title="Category:Ph.D.">Ph.D.</a>
<a href="/wiki/Category:Season_3_Characters" title="Category:S

In [13]:
# identify the type of tag to retrieve
link_tag = 'a'

# create a list with the links from the `<a>` tag
tag_list = []
for t in article.find_all(link_tag):
    tag_list.append(t.get('href'))

# List comprehension version:
# tag_list = [t.get('href') for t in article.find_all(link_tag)]

print('Size of \'tag_list\':', len(tag_list))
tag_list

Size of 'tag_list': 415


['https://static.wikia.nocookie.net/bigbangtheory/images/e/e7/Plag16.jpg/revision/latest?cb=20190506103936',
 'https://static.wikia.nocookie.net/bigbangtheory/images/f/f6/BarryKripke.png/revision/latest?cb=20110225223327',
 '/wiki/John_Ross_Bowie',
 '/wiki/Amy_Farrah_Fowler',
 '/wiki/Beverly_Hofstadter',
 '/wiki/The_Killer_Robot_Instability',
 '/wiki/The_Change_Constant',
 '/wiki/Season_2',
 '/wiki/Season_3',
 '/wiki/Season_4',
 '/wiki/Season_5',
 '/wiki/Season_6',
 '/wiki/Season_7',
 '/wiki/Season_8',
 '/wiki/Season_9',
 '/wiki/Season_10',
 '/wiki/Season_11',
 '/wiki/Season_12',
 '/wiki/The_Killer_Robot_Instability',
 '/wiki/Caltech',
 '/wiki/String_theory',
 '/wiki/Leonard_Hofstadter',
 '/wiki/Sheldon_Cooper',
 '/wiki/Leonard_Hofstadter',
 '/wiki/Howard_Wolowitz',
 '/wiki/Rajesh',
 '/wiki/Amy_Farrah_Fowler',
 '#Series_Overview',
 '#Season_1',
 '#Season_2',
 '#Season_3',
 '#Season_4',
 '#Season_5',
 '#Season_6',
 '#Season_7',
 '#Season_8',
 '#Season_9',
 '#Season_10',
 '#Season_11',
 

In [14]:
# keep only the links to the wiki itself
wiki_tag_list = []
for link in tag_list:
    if link is not None and link[:6] == '/wiki/':
        wiki_link = link[6:]
        wiki_tag_list.append(wiki_link)

# List comprehension:
# wiki_tag_list = [link[6:] for link in tag_list if link is not None and link[:6] == '/wiki/']

print('Size of \'wiki_tag_list\':', len(wiki_tag_list))
wiki_tag_list

Size of 'wiki_tag_list': 332


['John_Ross_Bowie',
 'Amy_Farrah_Fowler',
 'Beverly_Hofstadter',
 'The_Killer_Robot_Instability',
 'The_Change_Constant',
 'Season_2',
 'Season_3',
 'Season_4',
 'Season_5',
 'Season_6',
 'Season_7',
 'Season_8',
 'Season_9',
 'Season_10',
 'Season_11',
 'Season_12',
 'The_Killer_Robot_Instability',
 'Caltech',
 'String_theory',
 'Leonard_Hofstadter',
 'Sheldon_Cooper',
 'Leonard_Hofstadter',
 'Howard_Wolowitz',
 'Rajesh',
 'Amy_Farrah_Fowler',
 'Season_1',
 'Kripke_Krippler',
 'M.O.N.T.E.',
 'Caltech',
 'The_Killer_Robot_Instability',
 'Penny',
 'Howard_Wolowitz',
 'Penny',
 'The_Friendship_Algorithm',
 'Season_3',
 'The_Electric_Can_Opener_Fluctuation',
 'Sheldon%27s_office',
 'The_Cafeteria',
 'Leonard_Hofstadter',
 'Rajesh_Koothrappali',
 'Sheldon_Cooper',
 'President_Siebert',
 'The_Vengeance_Formulation',
 'Apartment_4A',
 'Zack_Johnson',
 'Stuart_Bloom',
 'LeVar_Burton',
 'Raj%27s_apartment',
 'The_Toast_Derivation',
 'Rajesh_Koothrappali',
 'Professor_Rothman',
 'Siri',
 'The_B

In [15]:
filter

filter

In [16]:
# create a filter for undesired links
filter  = '(%s)' % '|'.join([
    'Season_',
    'Category:',
    'File:',
    'Help:',
    'Portal:',
    'action=',
    'Special:',
    'Talk:'
])
# remove the links that are found in the filter
filtered_tag_list = []
for t in wiki_tag_list:
    if not re.search(filter, t):
        filtered_tag_list.append(t)

# filtered_tag_list = [t for t in wiki_tag_list if not re.search(filter, t)]
print('Size of \'filtered_tag_list\':', len(filtered_tag_list))
filtered_tag_list

Size of 'filtered_tag_list': 251


['John_Ross_Bowie',
 'Amy_Farrah_Fowler',
 'Beverly_Hofstadter',
 'The_Killer_Robot_Instability',
 'The_Change_Constant',
 'The_Killer_Robot_Instability',
 'Caltech',
 'String_theory',
 'Leonard_Hofstadter',
 'Sheldon_Cooper',
 'Leonard_Hofstadter',
 'Howard_Wolowitz',
 'Rajesh',
 'Amy_Farrah_Fowler',
 'Kripke_Krippler',
 'M.O.N.T.E.',
 'Caltech',
 'The_Killer_Robot_Instability',
 'Penny',
 'Howard_Wolowitz',
 'Penny',
 'The_Friendship_Algorithm',
 'The_Electric_Can_Opener_Fluctuation',
 'Sheldon%27s_office',
 'The_Cafeteria',
 'Leonard_Hofstadter',
 'Rajesh_Koothrappali',
 'Sheldon_Cooper',
 'President_Siebert',
 'The_Vengeance_Formulation',
 'Apartment_4A',
 'Zack_Johnson',
 'Stuart_Bloom',
 'LeVar_Burton',
 'Raj%27s_apartment',
 'The_Toast_Derivation',
 'Rajesh_Koothrappali',
 'Professor_Rothman',
 'Siri',
 'The_Beta_Test_Initiation',
 'The_Rothman_Disintegration',
 'Professor_Rothman',
 'President_Siebert',
 'The_Stag_Convergence',
 'The_Cooper/Kripke_Inversion',
 'Amy_Farrah_Fowle

In [17]:
# remove duplicates
unique_tag_list = list(set(filtered_tag_list))
print('Size of \'unique_tag_list\':', len(unique_tag_list))
unique_tag_list

Size of 'unique_tag_list': 195


['The_Tenure_Turbulence',
 'The_Big_Bang_Theory',
 'Bert_Kibbler',
 'The_Change_Constant',
 'Rajesh',
 'Simon_Helberg',
 'Stan_Lee',
 'Template_talk:Characters',
 'The_Toast_Derivation',
 'Dennis_Kim',
 'Kara_Luiz',
 'The_Celebration_Experimentation',
 'Mary_Cooper',
 'Judd_Hirsch',
 'Mark_Hamill',
 'Brent_Spiner',
 'George_Takei',
 'Mayim_Bialik',
 'The_Beta_Test_Initiation',
 'Steven_V._Silver',
 'The_Social_Group',
 'Lucy',
 'The_History_of_Everything',
 'The_Electric_Can_Opener_Fluctuation',
 'Bill_Prady',
 'Anthony_Del_Broccolo',
 'Colonel_Richard_Williams',
 'Eric_Gablehauser',
 'String_Theory',
 'The_Comic_Book_Store_Regeneration',
 'George_Cooper_Sr.',
 'Bill_Nye',
 'Stephanie_Barnett',
 'Courtney_Henggeler',
 'Meemaw',
 'Kevin_Smith',
 'Emily_Sweeney',
 'The_Bow_Tie_Asymmetry',
 'Mike_Rostenkowski',
 'Ramona_Nowitzki',
 'Johnny_Galecki',
 'Anthony_Rich',
 'The_Allowance_Evaporation',
 'Laurie_Metcalf',
 'Shamy',
 'Kripke',
 'David_Gibbs',
 'Kunal_Nayyar',
 'Caltech',
 'Howard_

In [18]:
# convert escaped sequences
unquoted_tag_list = [unquote(t) for t in unique_tag_list]
print('Size of \'unquoted_tag_list\':', len(unquoted_tag_list))
unquoted_tag_list

Size of 'unquoted_tag_list': 195


['The_Tenure_Turbulence',
 'The_Big_Bang_Theory',
 'Bert_Kibbler',
 'The_Change_Constant',
 'Rajesh',
 'Simon_Helberg',
 'Stan_Lee',
 'Template_talk:Characters',
 'The_Toast_Derivation',
 'Dennis_Kim',
 'Kara_Luiz',
 'The_Celebration_Experimentation',
 'Mary_Cooper',
 'Judd_Hirsch',
 'Mark_Hamill',
 'Brent_Spiner',
 'George_Takei',
 'Mayim_Bialik',
 'The_Beta_Test_Initiation',
 'Steven_V._Silver',
 'The_Social_Group',
 'Lucy',
 'The_History_of_Everything',
 'The_Electric_Can_Opener_Fluctuation',
 'Bill_Prady',
 'Anthony_Del_Broccolo',
 'Colonel_Richard_Williams',
 'Eric_Gablehauser',
 'String_Theory',
 'The_Comic_Book_Store_Regeneration',
 'George_Cooper_Sr.',
 'Bill_Nye',
 'Stephanie_Barnett',
 'Courtney_Henggeler',
 'Meemaw',
 'Kevin_Smith',
 'Emily_Sweeney',
 'The_Bow_Tie_Asymmetry',
 'Mike_Rostenkowski',
 'Ramona_Nowitzki',
 'Johnny_Galecki',
 'Anthony_Rich',
 'The_Allowance_Evaporation',
 'Laurie_Metcalf',
 'Shamy',
 'Kripke',
 'David_Gibbs',
 'Kunal_Nayyar',
 'Caltech',
 'Howard_

In [19]:
# convert underscore to space
spaced_tag_list = []
for tag in unquoted_tag_list:
    processed_tag = re.sub('_', ' ', tag)
    spaced_tag_list.append(processed_tag)

# spaced_tag_list = [re.sub('_', ' ', t) for t in unquoted_tag_list]
print('Size of \'tag_list\':', len(spaced_tag_list))
spaced_tag_list

Size of 'tag_list': 195


['The Tenure Turbulence',
 'The Big Bang Theory',
 'Bert Kibbler',
 'The Change Constant',
 'Rajesh',
 'Simon Helberg',
 'Stan Lee',
 'Template talk:Characters',
 'The Toast Derivation',
 'Dennis Kim',
 'Kara Luiz',
 'The Celebration Experimentation',
 'Mary Cooper',
 'Judd Hirsch',
 'Mark Hamill',
 'Brent Spiner',
 'George Takei',
 'Mayim Bialik',
 'The Beta Test Initiation',
 'Steven V. Silver',
 'The Social Group',
 'Lucy',
 'The History of Everything',
 'The Electric Can Opener Fluctuation',
 'Bill Prady',
 'Anthony Del Broccolo',
 'Colonel Richard Williams',
 'Eric Gablehauser',
 'String Theory',
 'The Comic Book Store Regeneration',
 'George Cooper Sr.',
 'Bill Nye',
 'Stephanie Barnett',
 'Courtney Henggeler',
 'Meemaw',
 'Kevin Smith',
 'Emily Sweeney',
 'The Bow Tie Asymmetry',
 'Mike Rostenkowski',
 'Ramona Nowitzki',
 'Johnny Galecki',
 'Anthony Rich',
 'The Allowance Evaporation',
 'Laurie Metcalf',
 'Shamy',
 'Kripke',
 'David Gibbs',
 'Kunal Nayyar',
 'Caltech',
 'Howard 

In [20]:
# order the list
spaced_tag_list.sort()
print('Size of \'spaced_tag_list\':', len(spaced_tag_list))
spaced_tag_list

Size of 'spaced_tag_list': 195


['Aarti Mann',
 'Adam West',
 'Alessandra Torresani',
 'Alex Jensen',
 'Alfred Hofstadter',
 'Alice Amter',
 'Althea Davis',
 'Amy Farrah Fowler',
 'Anthony Del Broccolo',
 'Anthony Rich',
 'Anu',
 'Apartment 4A',
 'Barenaked Ladies',
 'Bernadette Rostenkowski-Wolowitz',
 'Bert Kibbler',
 'Beverly Hofstadter',
 'Bill Nye',
 'Bill Prady',
 'Brent Spiner',
 'Brian George',
 'Brian Greene',
 'Brian Patrick Wade',
 'Brian Posehn',
 'Brian Thomas Smith',
 'Buzz Aldrin',
 'Caltech',
 'Carol Ann Susi',
 'Carrie Fisher',
 'Casey Sander',
 'Charlie Sheen',
 'Christine Baranski',
 'Chuck Lorre',
 'Chuck Lorre Productions',
 'Cinnamon',
 'Claire',
 'Colonel Richard Williams',
 'Courtney Henggeler',
 'Dan',
 'Dave Goetsch',
 'David Gibbs',
 'David Saltzberg',
 'Dean Norris',
 'Debbie Wolowitz',
 'Denise',
 'Dennis Kim',
 'Dimitri',
 'Dr. Pemberton',
 'Emily Sweeney',
 'Eric Gablehauser',
 'Eric Kaplan',
 'Fun with Flags',
 'George Cooper Jr.',
 'George Cooper Sr.',
 'George Smoot',
 'George Takei'

### Create a filter for unwanted types of articles

In [21]:
# remove the links that start with "The"
no_episodes_tag_list = []
for tag in spaced_tag_list:
    if not tag.startswith('The'):
        no_episodes_tag_list.append(tag)

#no_episodes_tag_list = [t for t in tag_list if not tag.startswith('The')]

print('Size of \'no_episodes_tag_list\':', len(no_episodes_tag_list))
no_episodes_tag_list

Size of 'no_episodes_tag_list': 164


['Aarti Mann',
 'Adam West',
 'Alessandra Torresani',
 'Alex Jensen',
 'Alfred Hofstadter',
 'Alice Amter',
 'Althea Davis',
 'Amy Farrah Fowler',
 'Anthony Del Broccolo',
 'Anthony Rich',
 'Anu',
 'Apartment 4A',
 'Barenaked Ladies',
 'Bernadette Rostenkowski-Wolowitz',
 'Bert Kibbler',
 'Beverly Hofstadter',
 'Bill Nye',
 'Bill Prady',
 'Brent Spiner',
 'Brian George',
 'Brian Greene',
 'Brian Patrick Wade',
 'Brian Posehn',
 'Brian Thomas Smith',
 'Buzz Aldrin',
 'Caltech',
 'Carol Ann Susi',
 'Carrie Fisher',
 'Casey Sander',
 'Charlie Sheen',
 'Christine Baranski',
 'Chuck Lorre',
 'Chuck Lorre Productions',
 'Cinnamon',
 'Claire',
 'Colonel Richard Williams',
 'Courtney Henggeler',
 'Dan',
 'Dave Goetsch',
 'David Gibbs',
 'David Saltzberg',
 'Dean Norris',
 'Debbie Wolowitz',
 'Denise',
 'Dennis Kim',
 'Dimitri',
 'Dr. Pemberton',
 'Emily Sweeney',
 'Eric Gablehauser',
 'Eric Kaplan',
 'Fun with Flags',
 'George Cooper Jr.',
 'George Cooper Sr.',
 'George Smoot',
 'George Takei'



---



---



> > > > > > > > > © 2021 Institute of Data


---



---



